In [15]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Model.SVM import SVC
from tensorflow.keras import datasets
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import cv2

In [16]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [17]:
X_train_flatten = X_train.astype('float32') / 255
X_test_flatten = X_test.astype('float32') / 255

In [18]:
X_train_resize = X_train_flatten.reshape(-1, 28 * 28)
X_test_resize = X_test_flatten.reshape(-1, 28 * 28)

print(X_train_resize.shape, X_test_resize.shape)

(60000, 784) (10000, 784)


In [ ]:
df_X_train = pd.DataFrame(X_train_resize)
df_X_test = pd.DataFrame(X_test_resize)
df_y_train = pd.DataFrame(y_train)
df_y_test = pd.DataFrame(y_test)

df_X_train.head()

# 🪄 | Data visualization

In [ ]:
plt.figure(figsize=(14,12))
for digit_image in range(0, 30):
    plt.subplot(7, 10, digit_image + 1)
    grid_data = df_X_train.iloc[digit_image].values.reshape(28, 28)
    plt.imshow(grid_data, interpolation= 'None', cmap= 'gray')
    plt.xticks([])
    plt.yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Visualizing the number of class and counts in the datasets

sns.countplot(df_y_train["label"])
plt.xlabel('Class')
plt.ylabel('Counts')
plt.title('Class Distribution')
plt.show()

# ☣ | Building model SVM

In [19]:
svc = SVC(learning_rate=0.001, no_of_iterations=100, lambda_parameter=0.0001)
svc.fit(X_train_resize, y_train)

In [20]:
accuracy = svc.score(X_test_resize, y_test)
print(f"\nAccuracy: {accuracy * 100:.3f}%\n")


Accuracy: 91.680%



In [21]:
y_predict = svc.predict(X_test_resize)
print(f'Gia tri du doan: {y_predict[:5]}')
print(f'Gia tri thuc: {y_test[:5]}')

Gia tri du doan: [7 2 1 0 4]
Gia tri thuc: [7 2 1 0 4]


In [22]:
test_accuracy = accuracy_score(y_test, y_predict)
print(f'Test Accuracy: {test_accuracy * 100:.3f}%')

Test Accuracy: 91.680%


In [23]:
confusion = confusion_matrix(y_test, y_predict)
confusion

array([[ 964,    0,    1,    2,    0,    3,    7,    2,    1,    0],
       [   0, 1117,    2,    2,    0,    1,    4,    1,    8,    0],
       [  11,   12,  904,   25,   10,    4,   13,   11,   36,    6],
       [   5,    0,   19,  921,    2,   21,    4,   11,   14,   13],
       [   1,    3,    2,    1,  901,    1,   10,    1,    6,   56],
       [  10,    3,    1,   31,   13,  778,   20,    4,   22,   10],
       [  12,    3,    3,    2,    4,   17,  915,    0,    2,    0],
       [   3,   11,   21,    6,    9,    1,    1,  934,    4,   38],
       [  14,   17,    6,   28,   15,   38,   13,    8,  824,   11],
       [   9,    8,    1,   15,   30,   11,    0,   16,    9,  910]],
      dtype=int64)

In [26]:
class_wine = classification_report(y_test, y_predict)
print(class_wine)

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       980
           1       0.95      0.98      0.97      1135
           2       0.94      0.88      0.91      1032
           3       0.89      0.91      0.90      1010
           4       0.92      0.92      0.92       982
           5       0.89      0.87      0.88       892
           6       0.93      0.96      0.94       958
           7       0.95      0.91      0.93      1028
           8       0.89      0.85      0.87       974
           9       0.87      0.90      0.89      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [25]:
image = cv2.imread('')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)

resize_image = cv2.resize(blurred_image, (28, 28))

plt.imshow(resize_image, cmap= 'gray')
plt.show()

flatten_image = resize_image.astype('float32') / 255
flatten_image = flatten_image.reshape(1, -1)

prediction = svc.predict(flatten_image)
print(prediction)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [27]:
joblib.dump(svc, r"D:\\Graduation_Project\\Model\\svm_digit_classifier.pkl")

['D:\\\\Graduation_Project\\\\Model\\\\svm_digit_classifier.pkl']